<a href="https://colab.research.google.com/github/VandanaAgarwal/cv_proj/blob/master/video_label_facerecog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install facenet-pytorch
!pip install mmcv

     |████████████████████████████████| 1.9MB 11.0MB/s 
     |████████████████████████████████| 245kB 12.8MB/s 
     |████████████████████████████████| 194kB 14.1MB/s 
  Created wheel for mmcv: filename=mmcv-1.2.0-cp36-cp36m-linux_x86_64.whl size=402638 sha256=850fab67c612733ed11ebf1f1fa60ae8e872d9bcd5676908d42773dbc48e709d
  Stored in directory: /root/.cache/pip/wheels/ab/0a/b5/263a057c5aeedf3e417ca001122b1c012400b4d6ead29e934a
Successfully built mmcv


In [2]:
!git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch

Cloning into 'facenet_pytorch'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1245 (delta 3), reused 3 (delta 1), pack-reused 1235
Receiving objects: 100% (1245/1245), 22.87 MiB | 39.50 MiB/s, done.
Resolving deltas: 100% (605/605), done.


In [3]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=2273099a9bf08784a11aba5e04466935dffc8f4384b381b315d0f96e9f486005
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [1]:
import numpy as np
import cv2
import os
import face_recognition
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

In [5]:
!mkdir /content/frame_files

In [6]:
!mkdir /content/data

In [7]:
!mkdir /content/train
!mkdir /content/train/child1
!mkdir /content/train/lahar
!mkdir /content/train/nikki
!mkdir /content/train/vipin
!mkdir /content/train/mummy
!mkdir /content/train/mahi

In [8]:
! rm /content/train/mahi/mahi/*.*

rm: cannot remove '/content/train/mahi/mahi/*.*': No such file or directory


In [9]:
!cp -r /content/drive/MyDrive/data1/train_images/mahi /content/train/.
!cp -r /content/drive/MyDrive/data1/train_images/lahar /content/train/.
!cp -r /content/drive/MyDrive/data1/train_images/vipin /content/train/.
!cp -r /content/drive/MyDrive/data1/train_images/mummy /content/train/.

In [10]:
! cp /content/drive/MyDrive/data1/videos/2.mp4 /content/data/vipin_2.mp4

In [2]:
input_files = '/content/data/' # all videos
output_files = '/content/frame_files/' # frame.jpg files, frame.csv files, output video with bboxes and labels
trainPath = '/content/train' # training data

encoding_known = []
names = []
fr_skip_count = 5
df_frames = pd.DataFrame(columns=['Frame_file', 'Face_x0', 'Face_x1', 'Face_y0', 'Face_y1', 'Name'])

In [3]:
for name in os.listdir(trainPath) :
    for filename in os.listdir(f'{trainPath}/{name}') :
        print('##', f'{trainPath}/{name}/{filename}')
        if '.jpg' not in filename and '.png' not in filename : continue
        image = face_recognition.load_image_file(f'{trainPath}/{name}/{filename}')
        faces = face_recognition.face_locations(image, model='cnn')
        e = face_recognition.face_encodings(image, faces)
        if e :
            encodings = e[0]
            encoding_known.append(encodings)
            names.append(name)
print(names)
print(len(names), len(encoding_known))

## /content/train/lahar/11.jpg
## /content/train/lahar/1.jpg
## /content/train/lahar/10.jpg
## /content/train/lahar/7.jpg
## /content/train/lahar/4.jpg
## /content/train/lahar/3.jpg
## /content/train/lahar/9.jpg
## /content/train/lahar/2.jpg
## /content/train/lahar/5.jpg
## /content/train/lahar/6.jpg
## /content/train/lahar/8.jpg
## /content/train/mahi/11.jpg
## /content/train/mahi/.ipynb_checkpoints
## /content/train/mahi/12.jpg
## /content/train/mahi/14.jpg
## /content/train/mahi/10.jpg
## /content/train/mahi/7.jpg
## /content/train/mahi/3.jpg
## /content/train/mahi/9.jpg


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 99 bytes but only got 74. Skipping tag 45059
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/JpegImagePlugin.py:796: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  "Image appears to be a malformed MPO file, it will be "


## /content/train/mahi/2.jpg
## /content/train/mahi/6.jpg
## /content/train/mahi/13.jpg
## /content/train/mahi/8.jpg
['lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'lahar', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi', 'mahi']
22 22


In [4]:
#FOR FACE RECOGNITION LIBRARY
def get_encodings(img) :
  faces_current = face_recognition.face_locations(img, model='cnn')
  encode_current = face_recognition.face_encodings(img, faces_current)
  return faces_current, encode_current


In [5]:
 def lblImg_facerecog(imgFile, frame_file) :
  #img = cv2.resize(imgFile, (480, 640), None, 0.5, 0.5)
  img = cv2.resize(imgFile, (800, 1000), None, 0.5, 0.5)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  faces_current, encode_current = get_encodings(img)
  print('no of faces->', len(faces_current), 'frame file --->', frame_file)

  names_found = []
  for encodeF, faceLoc in zip(encode_current, faces_current):
          matches = face_recognition.compare_faces(encoding_known, encodeF, tolerance=0.5)
          face_dis = face_recognition.face_distance(encoding_known, encodeF)
          print('face dist-->', face_dis)
          matched_index = np.argmin(face_dis)
          
          if matches[matched_index]:
                  name = names[matched_index].upper()
          else :
                  name = 'unknown'

          names_found.append(name)
          top, right, bottom, left = faceLoc

          df_frames.loc[len(df_frames.index)] = [frame_file, left, right, top, bottom, name]
          #face = img[top:bottom, left:right, :]
          top-=4; right+=4; left-=4; bottom+=4
          #cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 5)
          #cv2.putText(img, name, (left, top - 20), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)
          
  cv2.imwrite(frame_file, img)
  print(names_found, end='********************\n')


In [6]:
def FrameCapture(path, rotate_fr=False):
    base = os.path.basename(path)
    fname = os.path.splitext(base)[0]
    new_out_folder = f'{output_files}{fname}'
    if not os.path.exists(new_out_folder) :
      os.mkdir(new_out_folder)

    min_id = 0
    max_id = 0

    vidObj = cv2.VideoCapture(path)
    count = 0
    
    fr_files_list = []
    while True :
        success, image = vidObj.read()

        if not success : break
        count += 1

        if not count % fr_skip_count :
          if rotate_fr :
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
          frame_file = new_out_folder + "/frame%d.jpg" % count
          fr_files_list.append(frame_file)
          if not min_id : min_id = count
          max_id = count
          lblImg_facerecog(image, frame_file)
          
    print('{0} frames captured'.format(count))
    return min_id, max_id, fr_files_list

In [ ]:
print(encoding_known)

In [7]:
video_file = 'vipin_2.mp4'
min_fr_id, max_fr_id, fr_files_list = FrameCapture(input_files + video_file, rotate_fr=True)

#video_file = 'vipin_1.mp4'
#min_fr_id, max_fr_id, fr_files_list = FrameCapture(input_files + video_file)

#video_file = 'vipin_3.mp4'
#min_fr_id, max_fr_id, fr_files_list = FrameCapture(input_files + video_file)

no of faces-> 2 frame file ---> /content/frame_files/vipin_2/frame5.jpg
face dist--> [0.5035593  0.52876293 0.50278909 0.5736848  0.53770209 0.58113429
 0.6390879  0.53903703 0.63689522 0.53273342 0.50145398 0.69104033
 0.74513649 0.75158262 0.74523936 0.74186116 0.70886875 0.71504515
 0.64913508 0.65938336 0.70280502 0.69757113]
face dist--> [0.76015345 0.737003   0.75445529 0.75361117 0.73164233 0.76512788
 0.79035182 0.74031026 0.79279514 0.7192469  0.72078706 0.66748158
 0.58094044 0.62620996 0.6068517  0.76143849 0.70345202 0.62649271
 0.62495677 0.70693169 0.66375131 0.6844896 ]
['unknown', 'unknown']********************
no of faces-> 2 frame file ---> /content/frame_files/vipin_2/frame10.jpg
face dist--> [0.68093768 0.70959033 0.67842459 0.71071455 0.7109414  0.70129896
 0.76000479 0.70652463 0.75899335 0.72401327 0.65285279 0.660783
 0.67768797 0.71335548 0.70077503 0.70278493 0.64411405 0.67454788
 0.59008312 0.61910346 0.66313314 0.68427209]
face dist--> [0.72766528 0.6654046

In [ ]:
print(df_frames.head())

                                 Frame_file Face_x0  ... Face_y1     Name
0   /content/frame_files/vipin_2/frame5.jpg     667  ...     113  unknown
1   /content/frame_files/vipin_2/frame5.jpg     227  ...     377  unknown
2  /content/frame_files/vipin_2/frame10.jpg     667  ...     104  unknown
3  /content/frame_files/vipin_2/frame10.jpg     184  ...     377     MAHI
4  /content/frame_files/vipin_2/frame15.jpg     675  ...     104  unknown

[5 rows x 6 columns]


In [8]:
fname = os.path.splitext(video_file)[0]
new_out_folder = f'{output_files}{fname}'

csv_folder = new_out_folder
if not os.path.exists(csv_folder) :
  os.mkdir(csv_folder)
frame_csv = csv_folder + '/frames_1.csv'
df_frames.to_csv(frame_csv)

In [9]:
def commonAr(rect1, rect2) :
    x0_1, x1_1, y0_1, y1_1 = rect1
    x0_2, x1_2, y0_2, y1_2 = rect2

    x0 = x0_1 if x0_1 > x0_2 else x0_2
    x1 = x1_1 if x1_1 < x1_2 else x1_2

    y0 = y0_1 if y0_1 > y0_2 else y0_2
    y1 = y1_1 if y1_1 < y1_2 else y1_2

    print(x0, x1, y0, y1)
    area = 0 if x0>x1 or y0>y1 else (x1-x0) * (y1-y0)
    return area

In [10]:
def find_mising_names_before(df_fr) :
    df = df_fr.copy(deep=True)
    modify = False
    prev_id = None
    for frame_id in range(min_fr_id, max_fr_id+1, fr_skip_count) :
        if not prev_id :
            prev_id = frame_id
            continue

        curr_id = frame_id
        curr_fr_file = new_out_folder + "/frame%d.jpg" % curr_id
        df_curr = df[df['Frame_file'] == curr_fr_file]
        prev_fr_file = new_out_folder + "/frame%d.jpg" % prev_id
        df_prev = df[df['Frame_file'] == prev_fr_file]
        #print(curr_fr_file, '##', prev_fr_file, '@@')
        #input()
        #print('df_curr->', df_curr)
        #print('df_prev->', df_prev)
        if df_prev.empty :
          prev_id = frame_id
          continue
        if df_curr.empty : continue
        print('\n****************\n')
        print(prev_fr_file, curr_fr_file, end=' ')
        
        # USING CURRENT, CHANGE PREVIOUS
        for inx_curr, row_curr in df_curr.iterrows() :
            x0_c, x1_c, y0_c, y1_c = row_curr['Face_x0'], row_curr['Face_x1'], row_curr['Face_y0'], row_curr['Face_y1']
            pred_nm = row_curr['Name']
            print(pred_nm)
            
            if pred_nm == 'unknown' or pred_nm == '' :
                continue

            curr_area = (x1_c-x0_c) * (y1_c-y0_c)
            for inx_prev, row_prev in df_prev.iterrows() :
                prev_pred_nm = row_prev['Name']
                print('name in prev pred -->', prev_pred_nm)
                
                if prev_pred_nm != 'unknown' and prev_pred_nm != '' : continue
                print('CHECKING FOR OVERLAP....')
                
                x0_p, x1_p, y0_p, y1_p = row_prev['Face_x0'], row_prev['Face_x1'], row_prev['Face_y0'], row_prev['Face_y1']
                print('BOX1 --->', (x0_c, x1_c, y0_c, y1_c), 'BOX2 --->', (x0_p, x1_p, y0_p, y1_p))
                overlap = commonAr((x0_c, x1_c, y0_c, y1_c), (x0_p, x1_p, y0_p, y1_p))
                percent_overlap = (overlap / curr_area) * 100 if overlap else 0
                
                print('overlap -->', overlap, 'percent overlap -->', percent_overlap)
                if percent_overlap > 60 :
                    modify = True
                    print('\n\n** overlap found **\n')
                    query = f'Frame_file=="{prev_fr_file}" & Face_x0=={x0_p} & Face_x1=={x1_p} & Face_y0=={y0_p} & Face_y1=={y1_p}'
                    inxs = df.query(query).index.tolist()
                    df.loc[inxs, 'Name'] = pred_nm
                    break

        prev_id = frame_id
    return df, modify
    

In [11]:
count = 1
while True :
  file_orig = csv_folder + f'/frames_{count}.csv'
  file_new = csv_folder + f'/frames_{count+1}.csv'

  print('\n\n\n#######################')
  print(file_orig, '@@@@@@@@@@@@@@', file_new)
  input('enter a key...')

  df_1 = pd.read_csv(file_orig)
  df_new, modify = find_mising_names_before(df_1)
  df_new.to_csv(file_new)
  count += 1
  if not modify : break
  




#######################
/content/frame_files/vipin_2/frames_1.csv @@@@@@@@@@@@@@ /content/frame_files/vipin_2/frames_2.csv
enter a key...

****************

/content/frame_files/vipin_2/frame5.jpg /content/frame_files/vipin_2/frame10.jpg unknown
unknown

****************

/content/frame_files/vipin_2/frame10.jpg /content/frame_files/vipin_2/frame15.jpg unknown
unknown

****************

/content/frame_files/vipin_2/frame15.jpg /content/frame_files/vipin_2/frame20.jpg MAHI
name in prev pred --> unknown
CHECKING FOR OVERLAP....
BOX1 ---> (127, 269, 235, 377) BOX2 ---> (675, 757, 22, 104)
675 269 235 104
overlap --> 0 percent overlap --> 0
name in prev pred --> unknown
CHECKING FOR OVERLAP....
BOX1 ---> (127, 269, 235, 377) BOX2 ---> (141, 283, 206, 348)
141 269 235 348
overlap --> 14464 percent overlap --> 71.73179924618131


** overlap found **

unknown

****************

/content/frame_files/vipin_2/frame20.jpg /content/frame_files/vipin_2/frame25.jpg MAHI
name in prev pred --> MAHI

In [12]:
def find_mising_names_after(df_fr) :
    df = df_fr.copy(deep=True)
    modify = False
    prev_id = None
    for frame_id in range(min_fr_id, max_fr_id+1, fr_skip_count) :
        if not prev_id :
            prev_id = frame_id
            continue

        curr_id = frame_id
        curr_fr_file = new_out_folder + "/frame%d.jpg" % curr_id
        df_curr = df[df['Frame_file'] == curr_fr_file]
        prev_fr_file = new_out_folder + "/frame%d.jpg" % prev_id
        df_prev = df[df['Frame_file'] == prev_fr_file]
        if df_prev.empty :
          prev_id = frame_id
          continue
        if df_curr.empty : continue
        print('\n****************\n')
        print(prev_fr_file, curr_fr_file, end=' ')
                
        # USING PREVIOUS, CHANGE CURRENT
        for inx_prev, row_prev in df_prev.iterrows() :
            x0_p, x1_p, y0_p, y1_p = row_prev['Face_x0'], row_prev['Face_x1'], row_prev['Face_y0'], row_prev['Face_y1']
            pred_nm = row_prev['Name']
            print(pred_nm)
            
            if pred_nm == 'unknown' or pred_nm == '' :
                continue

            prev_area = (x1_p-x0_p) * (y1_p-y0_p)
            for inx_curr, row_curr in df_curr.iterrows() :
                curr_pred_nm = row_curr['Name']
                print('name in curr pred -->', curr_pred_nm)
                
                if curr_pred_nm != 'unknown' and curr_pred_nm != '' : continue
                print('CHECKING FOR OVERLAP....')
                
                x0_c, x1_c, y0_c, y1_c = row_curr['Face_x0'], row_curr['Face_x1'], row_curr['Face_y0'], row_curr['Face_y1']
                print('BOX1 --->', (x0_p, x1_p, y0_p, y1_p), 'BOX2 --->', (x0_c, x1_c, y0_c, y1_c))
                overlap = commonAr((x0_p, x1_p, y0_p, y1_p), (x0_c, x1_c, y0_c, y1_c))
                percent_overlap = (overlap / prev_area) * 100 if overlap else 0
                
                print('overlap -->', overlap, 'percent overlap -->', percent_overlap)
                if percent_overlap > 60 :
                    modify = True
                    print('\n\n** overlap found **\n')
                    query = f'Frame_file=="{curr_fr_file}" & Face_x0=={x0_c} & Face_x1=={x1_c} & Face_y0=={y0_c} & Face_y1=={y1_c}'
                    inxs = df.query(query).index.tolist()
                    df.loc[inxs, 'Name'] = pred_nm
                    break

        prev_id = frame_id
    return df, modify

In [13]:
count

7

In [14]:
#count = 5
while True :
  file_orig = csv_folder + f'/frames_{count}.csv'
  file_new = csv_folder + f'/frames_{count+1}.csv'

  print('\n\n\n#######################')
  print(file_orig, '@@@@@@@@@@@@@@', file_new)
  input('enter a key...')

  df_1 = pd.read_csv(file_orig)
  df_new, modify = find_mising_names_after(df_1)
  
  df_new.to_csv(file_new)
  count += 1
  if not modify : break
  




#######################
/content/frame_files/vipin_2/frames_7.csv @@@@@@@@@@@@@@ /content/frame_files/vipin_2/frames_8.csv
enter a key...

****************

/content/frame_files/vipin_2/frame5.jpg /content/frame_files/vipin_2/frame10.jpg LAHAR
name in curr pred --> LAHAR
name in curr pred --> unknown
CHECKING FOR OVERLAP....
BOX1 ---> (667, 749, 31, 113) BOX2 ---> (184, 326, 235, 377)
667 326 235 113
overlap --> 0 percent overlap --> 0
unknown

****************

/content/frame_files/vipin_2/frame10.jpg /content/frame_files/vipin_2/frame15.jpg LAHAR
name in curr pred --> LAHAR
name in curr pred --> MAHI
unknown

****************

/content/frame_files/vipin_2/frame15.jpg /content/frame_files/vipin_2/frame20.jpg LAHAR
name in curr pred --> MAHI
name in curr pred --> LAHAR
MAHI
name in curr pred --> MAHI
name in curr pred --> LAHAR

****************

/content/frame_files/vipin_2/frame20.jpg /content/frame_files/vipin_2/frame25.jpg MAHI
name in curr pred --> MAHI
name in curr pred --> LA

In [15]:
res=(800, 1000)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec
out = cv2.VideoWriter(new_out_folder + '/test_vipin.mp4', fourcc, 20.0, res)

In [16]:
final_csv = csv_folder + f'/frames_{count}.csv'
df_final = pd.read_csv(final_csv)  #input_files + 'frames_7.csv')
for file_nm in fr_files_list :
  img = cv2.imread(file_nm)
  df_temp = df_final[df_final.Frame_file == file_nm]
  #print(file_nm)
  #print(df_temp)
  if not df_temp.empty :
    for inx, row in df_temp.iterrows() :
      left, right, top, bottom = row['Face_x0'], row['Face_x1'], row['Face_y0'], row['Face_y1']
      name = row['Name']

      cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 3)
      cv2.putText(img, name, (left, top - 20), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)
  out.write(img)

out.release()

In [ ]:
!rm /content/frame_files/*.jpg

**For pytorch code -> not using**

In [ ]:
video = mmcv.VideoReader(input_files + 'vipin_1.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')



In [ ]:
#FOR PYTORCH LIBRARY
from facenet_pytorch import MTCNN, InceptionResnetV1

# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=(800, 1000))class my_list(list) :
    def __init__(s, iterable) :
        x = [i+100 for i in iterable]
        super(my_list, s).__init__(x)

l = my_list([1,2,3])
print(l)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
from PIL import Image
def lblImg_pytorch(imgFile, frame_file) :
  img = cv2.resize(imgFile, (800, 1000), None, 0.5, 0.5)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  img = Image.fromarray(img)

  # Detect faces
  boxes, _ = mtcnn.detect(img)
  #if boxes : print('no of faces->', len(boxes))

  img = np.array(img)  
  if boxes is not None :
    names_found = []
    for box in boxes:
          #draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
          left, top, right, bottom = box.tolist()
          left = int(left)
          right = int(right)
          top = int(top)
          bottom = int(bottom)
          
          name = 'unknown'
          names_found.append(name)
          
          #df_frames.loc[len(df_frames.index)] = [frame_file, left, right, top, bottom, '']
          cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 5)
          cv2.putText(img, name, (left, top - 20), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)

  cv2.imwrite(frame_file, img)

In [ ]:
# TEST lblImg_pytorch FUNCTION
img = cv2.imread('/content/data/p1.jpg')
lblImg_pytorch(img, '/content/data/temp.jpg')